<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=200 height=200 align=left class="saturate" >

<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=6>
    Artificial Intelligence <br><br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Spring 2023<br><br>
<font color=3C99D size=5>
    Practical Assignment 2 - CSP <br>
<font color=696880 size=3>

____

# Personal Data

In [1]:
# Set your student number
student_number = '401011151'
name = 'Sara'
last_name = 'Karimi'

# Rules
- Make sure that all of your cells can be run perfectly. 

# Q1: Cryptarithmetic Puzzle (100 Points)

<font size=4>
Author: Yalda Shabanzadeh
<br/>
<font color=red>
Please run all the cells.
</font>
</font>
<br/>
</div>

## Problem Definition

The Cryptarithmetic Puzzle is a puzzle where the digits of some numbers are replaced with letters to make a mathematical equation. Each letter represents a unique digit from 0 to 9. The puzzle has the following constraints:

- Each letter can be replaced by a digit from 0 to 9, but no digit can be used twice.
- Each letter must be replaced by the same digit throughout the puzzle.
- The leftmost digit of each number cannot be zero.

### Example
Here's an example puzzle:

```
  TWO
+ TWO
------
 FOUR
```

In this puzzle, we need to replace each letter with a digit from 0 to 9 to make the equation true.

## Note
- You can use any arbitrary CSP algorithm to solve this question.
- Allowed algorithms:
BackTracking, AC3, AC3-LCV, AC3-MRV, AC3-LCV-MRV

> Explain how you consider this problem as a CSP (5 points)


<font color='cyan'>
if we consider each of the letters as variables in the inputs, we can model the problem by assigning multiple constraints on each of the variables including unary constraint of not being zero on first letters and multiary constraint on variables on same column to make their sum match the result. reaching an assignment that satisfies all these constraints is the goal and we don't need any follow-up optimization to the assignments so using CSP is ideal for the problem.
</font>

## Input
  - The first line is $n$ (count of strings to add up together).
  - In the following $n$ lines, the $i$ th line ($1 \le i \le N$) is $i$ th string.
  - In the next line, $result$, the sum of strings is given.
  - **Note**: your code should be case insensitive to pass the tests (there is no difference between `send` and `SEND`).
 
$$1 \le N \le 50$$

### Sample Input
This sample describes the previous example.
```
2
TWO
TWO
FOUR
```

## Output
for each given charachter as input, print it's corresponding number:
  - **Note**: order is not important.

### Sample Output
The solution to the example above is:
```
F 1
O 4
R 8
T 7
U 6
W 3
```

## Your code
**Note:** It's OK to change the signature for the given functions and the given structure is just a suggestion to help you with the implementation. If you tried to use another structure, the total score of that part won't be changed.

### Define Constraint (20 points)

In this part, write the satisfaction part of the constraint of this problem.
You can add any argument you want to `__init__` function.


- Check if all letters have a different value.
- Check if the first letter of each word is not 0.
- Check if the sum of the values of the letters 
  in the words is equal to the value of the letters
  in the result word. (Note that if the assignment 
  does not have all variables, you can not check this constraint, so return true in this case)
        

In [2]:
from typing import Dict, List


class PuzzleConstraint:
    def __init__(self, 
                 variables: List[str],
                 operands: List[str],
                 result: str) -> None:
        self.variables = variables
        self.operands = operands
        self.expected_result = result


    def satisfied(self, assignment: Dict[str, int]) -> bool:
        """
        :param assignment: dict[str, int] a dictionary of variable (str)
                           and it's correspond value (int)
        :return: check if the assignment satisfies the constraints
        """
        # Check if all letters have a different value
        if len(assignment.keys()) != len(set(assignment.values())):
            return False
        
        # Check if the first letter of the word is not 0.
        for op in self.operands:
            if assignment[op[0]] == 0:
                return False
        if assignment[self.expected_result[0]] == 0:
            return False
        
        # Check if the sum of the values of the letters 
        # in the words is equal to the value of the letters
        # in the result word.
        if len(assignment) < len(self.variables):
            return False
        operands = []
        for op in self.operands:
            operand = 0
            for var in op:
                operand = operand * 10 + assignment[var]
            operands.append(operand)
        result = 0
        for var in self.expected_result:
            result = result * 10 + assignment[var]
        if sum(operands) != result:
            return False
        return True

### Define a CSP class (50 points)

You can add functions you'll need to this class.

In [3]:
class CSP:
    def __init__(self, variables, domains) -> None:
        self.variables = variables  # variables to be constrained
        self.domains = domains      # domain of each variable
        self.constraints = {}
        for variable in self.variables:
            self.constraints[variable] = []
            if variable not in self.domains:
                raise LookupError("Every variable should have a domain assigned to it.")

    def add_constraint(self, constraint: PuzzleConstraint) -> None:
        """
        Add a constraint to the problem 
        :param constraint: PuzzleConstraint
        :return: None
        """
        self.constraint = constraint

    def consistent(self, assignment: Dict[str, int]) -> bool:
        """
        Check if the value assignment is consistent
        :param variable: str
        :param assignment: dict[str, int] a dictionary of variable (str)
                           and it's correspond value (int)
        """
        return self.constraint.satisfied(assignment)

    def backtracking_search(self, assignment={}):
        unassigned_vars = set(self.variables) - set(assignment.keys())
        if len(unassigned_vars) == 0:
            return self.consistent(assignment)
        var = sorted(list(unassigned_vars))[0]
        for val in self.domains[var]:
            assignment[var] = val
            ok = self.backtracking_search(assignment)
            if ok:
                return True
        del assignment[var]
        return False

### Build CSP and solve it! (20 points)

complete these functions for testing.

In [4]:
def get_problem(operands: List[str], result: str) -> CSP:
    """
    Define the CSP problem
    
    :param operands: List[str]
    :param result: str
    :return: return the CSP
    """
    variables = set()
    for op in operands:
        for c in op:
            variables.add(c)
    for c in result:
        variables.add(c)
    variables = sorted(list(variables))
    csp = CSP(variables, domains=dict([[v, [i for i in range(0, 10)]] for v in variables]))
    csp.add_constraint(PuzzleConstraint(variables, operands, result))
    return csp

In [5]:
def backtracking_search(csp: CSP):
    """
    solve CSP

    :param csp: CSP
    :return: return the founded solution
    """
    sol = {}
    ok = csp.backtracking_search(sol)
    if ok:
        return '\n'.join(sorted([f'{k.upper()} {sol[k]}' for k in sol])) + '\n'
    return None

## Test

**Do not change the following cell.**

**Note**: 5 tests will be added after deadline. (5 points).

In [6]:
import helper.test_q2 as q2
import time

TIME_LIMIT = 3

tests = q2.get_all_tests(prefix='q2_')
tests_passed = 0
for test in tests:
    n, n_next_lines, result = q2.scan_test_input(test)
    csp = get_problem(n_next_lines, result)
    start_time = time.time()
    result = backtracking_search(csp)
    total_time = time.time() - start_time
    if q2.is_result_valid(test, result) and total_time < TIME_LIMIT:
        tests_passed += 1
    else:
        print(f'test {test} failed. time elapsed= {total_time}')
print(f'Score = {tests_passed / len(tests) * 100}%')

Score = 100.0%
